# Masking Data With Py-ART Gatefilters

In [ ]:
# This notebook gives examples on how to use a Py-ART GateFilter object to mask data.

In [ ]:
# First import needed modules.
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pyart

In [ ]:
# Read the radar file.
radar = pyart.io.read("sgpxsaprcmacsurI5.c1.20171004.203018.nc")

In [ ]:
# See what moments are present.
radar.fields.keys()

In [ ]:
# When plotting, we can see second trip echoes present as well as nonhydrometeor echoes.
fig = plt.figure(figsize=[8, 8])
display = pyart.graph.RadarMapDisplay(radar)
display.plot_ppi_map(
    "reflectivity",
    sweep=2,
    resolution="50m",
    vmin=-8,
    vmax=64,
    min_lon=-98.25,
    max_lon=-96.75,
    min_lat=35.75,
    max_lat=37.25,
    projection=ccrs.PlateCarree(),
)
plt.show()

In [ ]:
# When both displaying and analyzing such data, it is desirable to remove such echoes.
# Py-ART provides the GateFilter class for doing such filtering.
gatefilter = pyart.filters.GateFilter(radar)

In [ ]:
# The below example shows how you can use a gatefilter to exclude values where gate_id is
# equal to a certain value. (You can exclude values based on conditions from multiple
# fields. In this example, we are excluding regions identified as second trip echoes
# (gate_id = 0), nonhydrometeors (gate_id = 1), and clutter (gate_id = 2).
gatefilter.exclude_equal("gate_id", 0)
gatefilter.exclude_equal("gate_id", 3)
gatefilter.exclude_equal("gate_id", 5)

In [ ]:
# The gatefilter can be then be specified as an input to plot_ppi_map()
# As you can see, much cleaner with the clutter removed.
fig = plt.figure(figsize=[8, 8])
display = pyart.graph.RadarMapDisplay(radar)
display.plot_ppi_map(
    "reflectivity",
    sweep=2,
    resolution="50m",
    vmin=-8,
    vmax=64,
    min_lon=-98.25,
    max_lon=-96.75,
    min_lat=35.75,
    max_lat=37.25,
    projection=ccrs.PlateCarree(),
    gatefilter=gatefilter,
)
plt.show()

In [ ]:
# Let's reset the gatefilter and try another method.
gatefilter = pyart.filters.GateFilter(radar)

In [ ]:
# Lets remove reflectivity values below a threshold.
gatefilter.exclude_below("reflectivity", 0)

In [ ]:
# Here we see reflectivity values below zero masked.
fig = plt.figure(figsize=[8, 8])
display = pyart.graph.RadarMapDisplay(radar)
display.plot_ppi_map(
    "reflectivity",
    sweep=2,
    resolution="50m",
    vmin=-8,
    vmax=64,
    min_lon=-98.25,
    max_lon=-96.75,
    min_lat=35.75,
    max_lat=37.25,
    projection=ccrs.PlateCarree(),
    gatefilter=gatefilter,
)
plt.show()

In [ ]:
# The structure of the gatefilter includes boolean arrays of the same shape as the radar
# fields that tell you whether a gate is excluded (gatefilter.gate_excluded) or included
# (gatefilter.gate_included). These can be used to do calculations on the data that
# include the filter. For example, if we want to get the mean reflectivity of each sweep, we can do:
refl_array = np.ma.masked_where(
    gatefilter.gate_included == False, radar.fields["reflectivity"]["data"]
)
reflectivity_w_mask = [refl_array[x].mean() for x in radar.iter_slice()]
reflectivity = [
    radar.fields["reflectivity"]["data"][x].mean() for x in radar.iter_slice()
]
print(reflectivity)
print(reflectivity_w_mask)